In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip drive/MyDrive/Colab_Notebooks_2/data/AWA2.zip

Archive:  drive/MyDrive/Colab_Notebooks_2/data/AWA2.zip
 extracting: AWA2/att_splits.mat     
 extracting: AWA2/binaryAtt_splits.mat  
 extracting: AWA2/res101.mat         
 extracting: AWA2/testclasses.txt    
 extracting: AWA2/trainclasses1.txt  
 extracting: AWA2/trainclasses2.txt  
 extracting: AWA2/trainclasses3.txt  
 extracting: AWA2/trainvalclasses.txt  
 extracting: AWA2/valclasses1.txt    
 extracting: AWA2/valclasses2.txt    
 extracting: AWA2/valclasses3.txt    


In [7]:
!mv AWA2 drive/MyDrive/Colab_Notebooks_2/data/

In [8]:
import numpy as np
import argparse
from scipy import io, spatial
import time
from random import shuffle
import random
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
 

res101 = io.loadmat('drive/MyDrive/Colab_Notebooks_2/data/AWA2/res101.mat')
att_splits=io.loadmat('drive/MyDrive/Colab_Notebooks_2/data/AWA2/att_splits.mat')

In [9]:
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'
 
feat = res101['features']
# Shape -> (dxN)
X_train = feat[:, np.squeeze(att_splits[train_loc]-1)]
X_val = feat[:, np.squeeze(att_splits[val_loc]-1)]
X_test = feat[:, np.squeeze(att_splits[test_loc]-1)]


In [13]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(2048, 20218)
(2048, 9191)
(2048, 7913)


In [15]:
# labels for splits
labels = res101['labels']
labels_train = np.squeeze(labels[np.squeeze(att_splits[train_loc]-1)])
labels_val = np.squeeze(labels[np.squeeze(att_splits[val_loc]-1)])
labels_test = np.squeeze(labels[np.squeeze(att_splits[test_loc]-1)])

(20218,)

In [16]:
# labels divided to seen and unseen
train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
test_labels_unseen = np.unique(labels_test)

train_labels_seen = np.array([x-1 for x in train_labels_seen])

val_labels_unseen= np.array([x-1 for x in val_labels_unseen])

test_labels_unseen = np.array([x-1 for x in test_labels_unseen])

In [19]:
# semantic embeddings of the labels?
i=0
for labels in train_labels_seen:
  labels_train[labels_train == labels] = i    
  i+=1
 
j=0
for labels in val_labels_unseen:
  labels_val[labels_val == labels] = j
  j+=1
 
k=0
for labels in test_labels_unseen:
  labels_test[labels_test == labels] = k
  k+=1
 
sig = att_splits['att']
  # Shape -> (Number of attributes, Number of Classes)
train_sig = sig[:, train_labels_seen]
val_sig = sig[:, val_labels_unseen]
test_sig = sig[:, test_labels_unseen]

(85, 10)

In [20]:
sorted_ind =  np.concatenate([val_labels_unseen , train_labels_seen], axis=0)
sorted_ind=np.sort(sorted_ind)
train_sig_fin = sig[:, sorted_ind]

testClasses = test_labels_unseen
trainClasses =  np.concatenate([train_labels_seen , val_labels_unseen], axis=0)

sorted_ind_all =  np.concatenate([val_labels_unseen , train_labels_seen], axis=0)
sorted_ind_all =  np.concatenate([sorted_ind_all , test_labels_unseen], axis=0)
sorted_ind_all=np.sort(sorted_ind_all)

In [23]:
trainClasses

array([ 0,  1,  2,  5,  7,  9, 10, 12, 13, 16, 17, 18, 21, 26, 27, 31, 32,
       34, 35, 36, 37, 41, 42, 43, 44, 45, 48,  3,  4, 11, 14, 15, 19, 20,
       24, 25, 28, 38, 39, 47])

In [24]:
# Loads features
X = res101['features'].transpose()
Y_temp = res101['labels']
Y_temp=Y_temp.transpose()
Y_temp=Y_temp[0]
Y = np.array([x-1 for x in Y_temp])
Y =Y.astype(np.int32).transpose()
#Y1 =np.array(Y1.tolist())
ATTR =  att_splits['att']
att =  att_splits['att']
att=np.transpose(att)
noExs = X.shape[0]

In [25]:

trainDataX = []
trainDataLabels = []
trainDataAttrs = []

testDataX = []
testDataLabels = []
testDataAttrs = []



for ii in range(0,noExs):
    if(Y[ii] in trainClasses):
        trainDataX = trainDataX + [X[ii]]
        trainDataLabels = trainDataLabels + [Y[ii]]
        trainDataAttrs = trainDataAttrs + [att[Y[ii]]]
    elif(Y[ii] in testClasses):
        #print(str(Y[ii])  + "  is in   " + str(testClasses))
        testDataX = testDataX + [X[ii]]
        testDataLabels = testDataLabels + [Y[ii]]
        testDataAttrs = testDataAttrs + [att[Y[ii]]]
    else:
        print('Fatal Error... Please check code/data')
       
       
trainDataX = np.array(trainDataX)
trainDataLabels = np.array(trainDataLabels)
trainDataAttrs = np.array(trainDataAttrs)

testDataX = np.array(testDataX)
testDataLabels = np.array(testDataLabels)
testDataAttrs = np.array(testDataAttrs)


In [40]:
import pandas as pd
print(f'train data shape {trainDataX.shape}')
print(f'train data labels shape {trainDataLabels.shape}') # no one hot encoded
print(f'unique train data labels {pd.Series(trainDataLabels).unique()}') 
print(f'train data auxiliary attributes (embeddings of the labels) {trainDataAttrs.shape}') 


train data shape (29409, 2048)
train data labels shape (29409,)
unique train data labels [ 0  3 36 32 24 45 48  4 39 18 21  7 38 19  1 25 13 17  2 14 42 11 15 43
 35 20  5 41 44 28 47 27  9 10 16 26 12 34 31 37]
train data auxiliary attributes (embeddings) (29409, 85)


In [42]:
trainDataAttrs[trainDataLabels==0] # same embeddings for one label (regardless the particular input image)

array([[-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501],
       [-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501],
       [-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501],
       ...,
       [-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501],
       [-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501],
       [-0.00375358, -0.00375358, -0.00375358, ...,  0.00882092,
         0.03640974,  0.03145501]])

In [44]:
import pandas as pd
print(f'test data shape {testDataX.shape}')
print(f'test data labels shape {testDataLabels.shape}') # no one hot encoded
print(f'unique test data labels {pd.Series(testDataLabels).unique()}') 
print(f'test data auxiliary attributes (embeddings of the labels) {testDataAttrs.shape}') 


test data shape (7913, 2048)
test data labels shape (7913,)
unique test data labels [29  8 40 49 30  6 33 23 22 46]
test data auxiliary attributes (embeddings of the labels) (7913, 85)
